<div style="display: flex; align-items: center; gap: 8px;">
    <a href="https://creativecommons.org/licenses/by/4.0/">
        <img src="https://licensebuttons.net/l/by/4.0/80x15.png" style="display: block;" />
    </a>
    <a href="https://opensource.org/licenses/MIT">
        <img src="https://img.shields.io/badge/License-MIT-green.svg" style="display: block;" />
    </a>
    <span style="margin-left: 10px; font-size: 13px;">
        &copy; Guillaume Rongier
    </span>
</div>

# How to use geomodpy?

In this notebook, you'll get a basic look at geomodpy's wrapper of GSLIB-like functions.

### Imports

In [ ]:
# To manipulate arrays and matrices
import numpy as np
# To manipulate labelled multi-dimensional arrays
import xarray as xr
# To create plots and visualizations
import matplotlib.pyplot as plt

# To create semivariogram models
from geomodpy.analysis.variography import Direction, VarioModel, VarioStruct
# To use GSLIB in Python
from geomodpy.wrapper.gslib import NSCORE, ExpVario, GAMV, SGSIM
# To plot semivariograms
from geomodpy.plotting import plot_semivariogram

## 1. Unconditional simulation

Let's start by randomly generating a single realization using sequential Gaussian simulation. You first need to instantiate a `SGSIM` object, which includes a semivariogram model as `VarioModel`, which itself includes a single spherical structure as `VarioStruct`:

In [ ]:
sgsim = SGSIM(
    vario_model=VarioModel(
        structures=VarioStruct(
            model='spherical',
            azimuth=45.,
            range_hmax=6000.,
            range_hmin=2000.,
        ),
    ),
    shape=(150, 100, 1),
    spacing=(100., 100., 1.),
    origin=(50., 50., 0.),
    n_realizations=1,
    seed=42,
)

In general, the input parameter names and order remains similar to GSLIB, and are all defined during object initialization. Some basic documentation is available as docstring:

In [ ]:
help(SGSIM)

And more information is available in [GSLIB's user guide](http://claytonvdeutsch.com/wp-content/uploads/2019/03/GSLIB-Book-Second-Edition.pdf).

Calling the actual Fortran function of GSLIB is always done by calling the function `run`:

In [ ]:
realization = sgsim.run()

Outputs are usually [pandas' dataframes](https://pandas.pydata.org/docs/user_guide/dsintro.html) or [xarray's datasets](https://docs.xarray.dev/en/stable/user-guide/data-structures.html), like here:

In [ ]:
realization

Stochastic simulations always return a dataset with four dimensions, the number of realizations followed by the cell indice dimensions, and three coordinates, x, y, and z. You can then use [xarray's plotting functions](https://docs.xarray.dev/en/stable/user-guide/plotting.html) to visualize the realization:

In [ ]:
realization.isel(Realization=0, W=0)['Variable'].plot(x='X', y='Y')

And use xarray functionalities in general to manipulate the dataset, for instance by converting the realization into porosity values: 

In [ ]:
realization['Variable'] = 0.02*realization['Variable'] + 0.1

In [ ]:
realization = realization.rename({'Variable': 'Porosity'})

Let's visualize the result with a cleaner plot:

In [ ]:
fig, ax = plt.subplots()
realization.isel(Realization=0, W=0)['Porosity'].plot(ax=ax, x='X', y='Y')
ax.set(xlabel='x (m)', ylabel='y (m)', title=None, aspect='equal');

## 2. Conditional simulations

Now you can use the first unconditional realization as ground truth to simulate some conditional realizations.

### 2.1. Data generation

Let's start by randomly extracting 80 data points from this first realization and convert the result to a pandas' dataframe:

In [ ]:
rng = np.random.default_rng(42)

In [ ]:
data = (realization
    .isel(
        U=xr.DataArray(rng.integers(realization.sizes['U'], size=80)),
        V=xr.DataArray(rng.integers(realization.sizes['V'], size=80)),
    )
    .to_dataframe()
    .reset_index(drop=True)
)

In [ ]:
data

You can use [pandas' functionalities](https://pandas.pydata.org/docs/user_guide/visualization.html) to visualize the data:

In [ ]:
ax = data.plot.scatter(x='X', y='Y', c='Porosity', xlabel='x (m)', ylabel='y (m)',
                       vmin=realization['Porosity'].min(), vmax=realization['Porosity'].max())
ax.set(xlim=(0., 15000.), ylim=(0., 10000.), aspect='equal');

### 2.2. Variography

Before simulating, you need a semivariogram model. First, let's normalize the data using GSLIB's normal score transformation:

In [ ]:
nscore = NSCORE(data, data_columns='Porosity')

In [ ]:
data, trans_table = nscore.run()

You got an extra column with the normalized porosity `NS:Porosity`:

In [ ]:
data

Now you can compute the experimental semivariogram of the normalized porosity, which you can compute along any number of directions:

In [ ]:
directions = []
for azimuth in [0., 22.5, 45., 67.5, 90., 112.5, 135.]:
    directions += [Direction(azimuth=azimuth,
                             azimuth_tolerance=22.5,
                             horizontal_bandwidth=2000.,
                             n_lags=10,
                             lag_separation_distance=500.)]

gamv = GAMV(data,
            coord_columns=('X', 'Y'),
            exp_vario=ExpVario('NS:Porosity', 'NS:Porosity', 1),
            direction=directions)
exp_vario = gamv.run()

You can then use geomodpy's function to make semivariogram visualization easier, and potentially adjust the number of lags and their separation distance:

In [ ]:
plot_semivariogram(directions, exp_vario)

Let's define the semivariogram model:

In [ ]:
vario_model = VarioModel(
    nugget_effect=0.,
    structures=VarioStruct(
        model='spherical',
        partial_sill=1.,
        azimuth=45.,
        range_hmax=5000.,
        range_hmin=2000.,
    ),
)

And compute its values along each direction:

In [ ]:
vario_model_values = vario_model.semivario(direction=directions)

To visualize and adjust the fit to the experimental semivariogram:

In [ ]:
plot_semivariogram(directions, exp_vario, vario_model_values)

### 2.3. Stochastic simulation

Now you're ready to generate some conditional realizations using Gaussian simulation. The principle is the same as with the first unconditional realization, except that you need to add the data and use the semivariogram model inferred from those data:

In [ ]:
sgsim = SGSIM(
    vario_model=vario_model,
    data=data,
    coord_columns=('X', 'Y'),
    data_columns='Porosity',
    shape=(150, 100, 1),
    spacing=(100., 100., 1.),
    origin=(50., 50., 0.),
    n_realizations=50,
    transform_data=True,
    seed=42,
    variable_name='Porosity',
)

In addition, you can automatically apply the normal score transformation to the data and the back transformation to the realizations by setting `transform_data` to true, and change the name of the variable in the outputed dataset using `variable_name` instead of renaming it in an extra step.

Let's generate the realizations:

In [ ]:
realizations = sgsim.run()

Visualize the first three:

In [ ]:
fig, axs = plt.subplots(ncols=3, sharey=True, figsize=(10, 2.5), layout='constrained')
for i, ax in enumerate(axs):
    im = realizations.isel(Realization=i, W=0)['Porosity'].plot(ax=ax, x='X', y='Y',
                                                                add_colorbar=False)
    ax.set(xlabel='x (m)', ylabel=None, title=None, aspect='equal');
fig.colorbar(im, ax=axs, label='Porosity')
axs[0].set(ylabel='y (m)');

And visualize the mean and standard deviation:

In [ ]:
fig, axs = plt.subplots(ncols=2, sharey=True, figsize=(10, 3), layout='constrained')
realizations['Porosity'].mean('Realization').plot(ax=axs[0], x='X', y='Y')
axs[0].set(xlabel='x (m)', ylabel='y (m)', title=None, aspect='equal')
realizations['Porosity'].std('Realization').plot(ax=axs[1], x='X', y='Y',
                                                 cbar_kwargs={'label': 'Standard deviation'})
axs[1].set(xlabel='x (m)', ylabel=None, title=None, aspect='equal');